In [49]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from Bio import SeqIO
import vcf
import io
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # For handling missing genotypes
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances # For distance calculation example
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform # Needed if using scipy's dendrogram directly, but not for plotly's ff
import plotly.figure_factory as ff
import plotly.express as px
import time
import traceback # For detailed error reporting
import warnings # To suppress specific warnings if needed


In [50]:
def summarize_variants_in_region(vcf_dict, contig, start, end):

    if not vcf_dict:
        print("Warning: Input vcf_dict is empty.")
        return pd.DataFrame(columns=['CHROM', 'POS', 'REF', 'ALT',
                                     'Absolute Count', 'Percentage (%)'])

    
    variant_occurrences = {}
    total_vcfs = len(vcf_dict)

    for vcf_name, record_list in vcf_dict.items():
        processed_in_vcf = set() # Track variants processed for this specific VCF
        for record in record_list:
            # Check if the record is within the specified region
            if record.CHROM == contig and start <= record.POS <= end:
                # Ensure ALT is handled correctly (list of strings)
                # Real pyvcf records have ALT as a list of Substitution objects
                try:
                    # Handle cases where ALT might be None or empty
                    if record.ALT is None:
                       alt_sequences = tuple()
                    else:
                       # Extract sequence string from each AltAllele/Substitution object
                       alt_sequences = tuple(str(alt.sequence) for alt in record.ALT if alt is not None)

                    # Create a unique identifier for the variant
                    variant_key = (
                        record.CHROM,
                        record.POS,
                        str(record.REF), # Ensure REF is a string
                        alt_sequences    # Tuple of ALT strings
                    )

                    # Avoid double counting if a variant appears multiple times in the same sample VCF (shouldn't happen in standard VCFs)
                    if variant_key not in processed_in_vcf:
                        # Initialize the set for this variant if it's the first time seeing it
                        if variant_key not in variant_occurrences:
                            variant_occurrences[variant_key] = set()
                        # Add the current VCF name to the set for this variant
                        variant_occurrences[variant_key].add(vcf_name)
                        processed_in_vcf.add(variant_key)

                except AttributeError as e:
                    print(f"Warning: Skipping record due to unexpected structure "
                          f"(maybe not a standard pyvcf Record?): {record}. Error: {e}")
                except Exception as e:
                     print(f"Warning: An unexpected error occurred processing record: {record}. Error: {e}")

    # Prepare data for the DataFrame
    summary_data = []
    if total_vcfs > 0:
        for variant_key, vcf_set in variant_occurrences.items():
            chrom, pos, ref, alts = variant_key
            absolute_count = len(vcf_set)
            percentage = (absolute_count / total_vcfs) * 100.0
            summary_data.append({
                'CHROM': chrom,
                'POS': pos,
                'REF': ref,
                'ALT': ','.join(alts) if alts else '.', 
                'Absolute Count': absolute_count,
                'Percentage (%)': round(percentage, 2) 
            })

    # Create and sort the DataFrame
    summary_df = pd.DataFrame(summary_data)

    if not summary_df.empty:
        summary_df = summary_df.sort_values(by=['CHROM', 'POS', 'REF', 'ALT']).reset_index(drop=True)
        # Ensure correct data types
        summary_df['POS'] = summary_df['POS'].astype(int)
        summary_df['Absolute Count'] = summary_df['Absolute Count'].astype(int)


    return summary_df

In [51]:
def parse_fasta_string_with_biopython(fasta_string):
    records = {}
    fasta_handle = io.StringIO(fasta_string) # str je priamo v subore a nie priamo v premennej

    for record in SeqIO.parse(fasta_handle, "fasta"): # Iteracia cez vsetky SeqIO zaznamy
        records[record.id] = record
    
    fasta_handle.close() # Zatvorenie "suboru" s Fastou
    return records

In [52]:
def parse_vcf_string_with_pyvcf(vcf_string):
    parsed_records = []
    vcf_handle = io.StringIO(vcf_string)
    vcf_reader = vcf.Reader(vcf_handle)
    for record in vcf_reader:
        parsed_records.append(record)
    vcf_handle.close()
    return parsed_records

In [53]:
contig_dropdown = widgets.Dropdown(
    options=[], # Začína s prázdnymi možnosťami
    description='Select Contig:',
    disabled=True, # Začína ako neaktívny
    style={'description_width': 'initial'}
)

In [54]:
# Vytvorenie widgetu pre reference
reference_uploader = widgets.FileUpload(
    accept='fa',  
    multiple=False,
    description='Reference'
)

output_area = widgets.Output() # Vytvorenie priestoru na výpis info o uploadoch (pre potreby vývoja)

def handle_reference_upload(change):
    global reference 
    reference = None
    with output_area:
        
        clear_output(wait=True) # Vycistenie priestoru na vypis
        contig_dropdown.options = []
        contig_dropdown.value = None
        contig_dropdown.disabled = True
        uploaded_reference = change['new'] # ukladanie uploadovanych dat

        if not uploaded_reference: # Kontrola na uploadnute data
            print("No file selected or upload cancelled.")
            return
        # Upload je zabalený v tuple 
        uploaded_reference = uploaded_reference[0] # Vytiahneme subor z tuple
        
        filename = uploaded_reference['name'] # Ulozenie mena suboru
        content_memory_position = uploaded_reference['content'] # Odkaz na miesto v pamati, kde je obsah ulozeny
        content = content_memory_position.tobytes() # Precitanie obsahu pamate v bytoch


        try:
            text_content = content.decode('utf-8') # Prelozenie bytoveho obsahu do textu. V text_content je teraz cely obsah nahranej referencky ako text.
            reference = parse_fasta_string_with_biopython(text_content) # Ulozenie uploadovanej fasty do SeqIO formatu v globalnej premennej reference
        except UnicodeDecodeError: # Ak nastala chyba pri preklade. To znamena, ze uploadnuty subor nebol korektny.
           print("\nCould not decode file as UTF-8 text.")

        if reference is not None: # Naplnenie dropdown menu
            print(f"Successfully loaded reference {filename}")

            
            # Extrahovanie ID kontigov zo SeqRecord objektov
            contig_names = list(reference.keys())
            if contig_names:
                contig_dropdown.options = contig_names # Nastavenie možností
                contig_dropdown.value = contig_names[0] # Nastavenie predvolenej hodnoty
                contig_dropdown.disabled = False # Aktivácia dropdownu
            else:
                print("V spracovaných záznamoch neboli nájdené žiadne ID kontigov.")
                # Dropdown ostáva neaktívny s prázdnymi možnosťami (už nastavené)

        else:
                print("Parsovanie zlyhalo (funkcia vrátila None). Dropdown nebude aktualizovaný.")
                # Dropdown ostáva neaktívny
    return

reference_uploader.observe(handle_reference_upload, names='value') # Nahranie akcie k upload widgetu.





In [55]:
# Vytvorenie widgetu pre vcfs
vcfs_uploader = widgets.FileUpload(
    accept='.vcf',  
    multiple=True,
    description='VCFs'
)

def handle_vcfs_upload(change):
    global vcfs
    vcfs = {}
    with output_area:
        
        clear_output(wait=True) # Vycistenie priestoru na vypis

        uploaded_vcfs = change['new'] # Sem sa nahraju uploadnute data

        if not uploaded_vcfs: # Kontrola na uploadnute data
            print("No file selected or upload cancelled.")
            return
        else:
            print(f"Successfully uploaded {len(uploaded_vcfs)} VCFs.")
        for uploaded_vcf in uploaded_vcfs: # Upload je zabalený v tuple 
            
            
            filename = uploaded_vcf['name'] # Ulozenie mena suboru
            content_memory_position = uploaded_vcf['content'] # Odkaz na miesto v pamati, kde je obsah ulozeny
            content = content_memory_position.tobytes() # Precitanie obsahu pamate v bytoch
            
            try:
                text_content = content.decode('utf-8') # Prelozenie bytoveho obsahu do textu. V text_content je teraz cely obsah nahranej referencky ako text.
                vcfs[filename] = parse_vcf_string_with_pyvcf(text_content)
            except UnicodeDecodeError: # Ak nastala chyba pri preklade. To znamena, ze uploadnuty subor nebol korektny.
               print("\nCould not decode file as UTF-8 text.")
    return

vcfs_uploader.observe(handle_vcfs_upload, names='value') # Nahranie akcie k upload widgetu.

In [56]:
def on_contig_change(change): # Zmeni hodnotu v start/stop poliach pri nastaveni kontigu
    """Updates start/end position widgets when dropdown selection changes."""
    selected_contig_id = change['new']

    record = reference[selected_contig_id]
    length = len(record.seq)

    start_pos_widget.value = 1 # 1-based indexovanie, zrozumitelnejsie pre uzivatelov, co budu hlavne biologovia
    end_pos_widget.value = length #1-based indexovanie, zrozumitelnejsie pre uzivatelov, co budu hlavne biologovia
    
    start_pos_widget.disabled = False
    end_pos_widget.disabled = False
    
    return

In [57]:
start_pos_widget = widgets.IntText(
    value=0, # Startovacia hodnota, pri spusteni instancie je 0
    description='Start Pos:',
    disabled=True, # Kym nebude mat zmysel nieco zadavat
    style={'description_width': 'initial'}
)

end_pos_widget = widgets.IntText(
    value=0, # Startovacia hodnota, pri spusteni instancie je 0
    description='End Pos (Length):',
    disabled=True, # Kym nebude mat zmysel nieco zadavat
    style={'description_width': 'initial'}
)

contig_dropdown.observe(on_contig_change, names='value') # Zmeny nastavaju pri zmene contigov

In [58]:
def handle_calculate_button(change):
    with output_area:
        clear_output(wait=True) # Vycistenie priestoru na vypis
        df = summarize_variants_in_region(vcfs, contig_dropdown.value, start_pos_widget.value, end_pos_widget.value)
        print(df.to_string())

In [59]:
calculate_button = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='success',
)

calculate_button.on_click(handle_calculate_button)

In [60]:
# --- Analysis Function ---
def run_pca_analysis_on_whole_genome(button_click_event):

    output_area.clear_output() # Clear previous results

    to_print = []

    with output_area: # Redirect print statements and display calls here
        start_time = time.time()
        to_print.append(f"Analysis started at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        to_print.append("="*50)
        to_print.append("Starting analysis using pre-loaded VCF data...")

        # 0. Check if input dictionary exists
        if 'vcfs' not in globals() or not isinstance(vcfs, dict) or not vcfs:
            to_print.append("Error: The required dictionary 'vcfs' was not found or is empty.")
            to_print.append("Please ensure it is defined and populated in a previous cell before running this script.")
            return
        else:
            to_print.append(f"   Found data for {len(vcfs)} samples in 'vcfs'.")

        try:
            samples = list(vcfs.keys())
            n_samples = len(samples)
            if n_samples == 0:
                to_print.append("Error: No samples found in 'vcfs'.")
                return

            # ==================================
            # === STEP 1: VARIANT FILTERING & MATRIX BUILDING ===
            # ==================================
            to_print.append("   Identifying unique variants passing filters in the specified region...")
            unique_variant_keys = set()
            variants_passing_filters_initial = 0

            for sample_id, records in vcfs.items():
                for record in records: # record is a pyvcf.model._Record
                    # MODIFIED FILTER: Include bi-allelic SNPs OR bi-allelic Indels
                    if (record.is_snp or record.is_indel) and len(record.ALT) == 1:
                        # Optional: check record.FILTER? ('PASS' or empty/None)
                        # if record.FILTER is None or len(record.FILTER) == 0 or 'PASS' in record.FILTER:
                            alt_str = str(record.ALT[0])
                            variant_key = (record.CHROM, record.POS, record.REF, alt_str)
                            unique_variant_keys.add(variant_key)
                            variants_passing_filters_initial += 1

            sorted_variant_keys = sorted(list(unique_variant_keys))
            variant_to_index = {var_key: i for i, var_key in enumerate(sorted_variant_keys)}
            n_variants = len(sorted_variant_keys)

            # UPDATED PRINT STATEMENT
            to_print.append(f"   Found {n_variants} unique, filtered variants (bi-allelic SNPs or Indels) in the region.")

            if n_variants == 0:
                to_print.append("Error: No suitable variants passed the filters in the specified region across all samples.")
                return

            # --- Build Genotype Matrix ---
            to_print.append(f"   Building genotype matrix ({n_samples} samples x {n_variants} variants)...")
            genotype_matrix = np.full((n_samples, n_variants), -1, dtype=np.float32) # Fill with missing indicator -1
            sample_to_index = {sample_id: i for i, sample_id in enumerate(samples)}

            # Pre-build variant lookup for potentially faster access
            variant_lookup = {}
            for sample_id, records in vcfs.items():
                for record in records:
                     # MODIFIED FILTER (Consistent with above)
                    if (record.is_snp or record.is_indel) and len(record.ALT) == 1:
                        chrom_pos = (record.CHROM, record.POS)
                        if chrom_pos not in variant_lookup: variant_lookup[chrom_pos] = {}
                        variant_lookup[chrom_pos][sample_id] = record

            # Populate the matrix
            # NOTE: PyVCF's call.gt_type typically returns 0, 1, 2 for bi-allelic SNPs AND Indels,
            # representing the count of the alternate allele. This encoding is used directly.
            for var_idx, var_key in enumerate(sorted_variant_keys):
                chrom, pos, ref, alt = var_key
                chrom_pos = (chrom, pos)
                if chrom_pos in variant_lookup:
                    records_at_pos = variant_lookup[chrom_pos]
                    for sample_id, record in records_at_pos.items():
                        if record.REF == ref and str(record.ALT[0]) == alt: # Match exact variant
                            sample_idx = sample_to_index[sample_id]
                            if len(record.samples) > 0:
                                call = record.samples[0] # Assuming single-sample VCFs in the dict values
                                gt_type = call.gt_type # 0, 1, 2, or None (count of ALT allele)
                                if gt_type is not None:
                                    genotype_matrix[sample_idx, var_idx] = float(gt_type)
                                # else: leave as -1 (missing)

            # --- Matrix Post-Processing (Imputation, Zero-Variance Check) ---
            to_print.append("\nPost-processing genotype matrix...")
            # Handle Missing Data (Imputation)
            genotype_matrix[genotype_matrix == -1] = np.nan # Convert placeholder to NaN
            missing_values_count = np.isnan(genotype_matrix).sum()
            if missing_values_count > 0:
                to_print.append(f"   Imputing {missing_values_count} missing genotypes using variant mean...")
                imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
                try:
                    imputed_genotypes = imputer.fit_transform(genotype_matrix)
                except ValueError as imp_err:
                    # Check if imputation failed because all values in a column are NaN
                    if "Input contains NaN" in str(imp_err) and np.isnan(genotype_matrix).all(axis=0).any():
                         to_print.append("   Warning: At least one variant site has missing data for all samples.")
                         to_print.append("   Trying imputation column by column or removing such columns.")
                         # Option 1: Impute column by column (more robust but slower)
                         # imputed_genotypes = genotype_matrix.copy()
                         # for j in range(imputed_genotypes.shape[1]):
                         #     col = imputed_genotypes[:, j].reshape(-1, 1)
                         #     if np.isnan(col).any() and not np.isnan(col).all():
                         #        imputer_col = SimpleImputer(missing_values=np.nan, strategy='mean')
                         #        imputed_genotypes[:, j] = imputer_col.fit_transform(col).flatten()
                         #     elif np.isnan(col).all():
                         #         print(f"   Column {j} (variant {sorted_variant_keys[j]}) is all NaN - cannot impute mean, will be removed later if variance is zero.")
                         #         # Leave as NaN - zero variance check will handle it

                         # Option 2: Remove columns that are all NaN before imputation (simpler)
                         to_print.append("   Removing columns that consist entirely of missing data before imputation.")
                         all_nan_cols = np.where(np.isnan(genotype_matrix).all(axis=0))[0]
                         if len(all_nan_cols) > 0:
                              to_print.append(f"   Indices of columns removed (all NaN): {all_nan_cols}")
                              # Keep track of which variants remain
                              original_indices = np.arange(genotype_matrix.shape[1])
                              valid_cols_indices = np.delete(original_indices, all_nan_cols)
                              if len(valid_cols_indices) == 0:
                                   to_print.append("Error: All variant columns consisted entirely of missing data. Cannot proceed.")
                                   return
                              genotype_matrix_filtered_nan = genotype_matrix[:, valid_cols_indices]
                              # Update sorted_variant_keys and variant_to_index if needed later (depends on downstream use)
                              sorted_variant_keys = [sorted_variant_keys[i] for i in valid_cols_indices]
                              variant_to_index = {var_key: i for i, var_key in enumerate(sorted_variant_keys)}
                              n_variants = len(sorted_variant_keys) # Update variant count
                              to_print.append(f"   Proceeding with {n_variants} variants after removing all-NaN columns.")
                         else:
                              genotype_matrix_filtered_nan = genotype_matrix # No columns were all NaN

                         # Now impute the filtered matrix
                         if np.isnan(genotype_matrix_filtered_nan).any(): # Check if any NaNs remain
                             imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
                             imputed_genotypes = imputer.fit_transform(genotype_matrix_filtered_nan)
                         elif genotype_matrix_filtered_nan.shape[1] > 0: # No NaNs left, but check if matrix is not empty
                             imputed_genotypes = genotype_matrix_filtered_nan
                         else: # Matrix became empty after removing all-NaN columns
                             to_print.append("Error: No variants remaining after removing columns with only missing data.")
                             return

                    else: # Different imputation error
                        to_print.append(f"Error during imputation: {imp_err}. Cannot proceed.")
                        traceback.print_exc()
                        return
            else:
                to_print.append("   No missing genotypes found.")
                imputed_genotypes = genotype_matrix # No imputation needed

            # Check for variants with zero variance AFTER imputation
            to_print.append("   Checking for variants with zero variance...")
            if imputed_genotypes.shape[1] > 0:
                variant_stds = np.nanstd(imputed_genotypes, axis=0) # Use nanstd for safety
                variant_stds = np.nan_to_num(variant_stds, nan=0.0) # Replace potential NaNs (if column was all NaN and wasn't removed) with 0
                non_zero_var_indices = np.where(variant_stds > 1e-6)[0]
                zero_var_indices = np.where(variant_stds <= 1e-6)[0]
                n_total_variants_after_imputation = imputed_genotypes.shape[1]
                n_zero_variance = len(zero_var_indices)
                n_non_zero_variance = len(non_zero_var_indices)
                to_print.append(f"   Found {n_non_zero_variance} variants with non-zero variance out of {n_total_variants_after_imputation}.")
            else:
                to_print.append("Error: No variants available in the matrix after initial filtering and NaN handling.")
                return

            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            #%%% BRANCH BASED ON VARIANCE PRESENCE %%%%%
            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

            # --- Handle Case: NO Variance Found ---
            if n_non_zero_variance == 0:
                to_print.append("\n" + "="*50)
                to_print.append("RESULT: All variants show NO VARIATION across samples (after imputation).")
                to_print.append("Samples are genetically identical based on the selected bi-allelic SNPs/Indels.")
                to_print.append("="*50)

                # ==============================================
                # === STEP 2: DISTANCE CALCULATION (No Variance Case) ===
                # ==============================================
                to_print.append("\n--- Pairwise Euclidean Distances ---")
                # Matrix should have low/zero distances if no variance
                if imputed_genotypes.shape[1] > 0:
                    try:
                        # Use the original imputed matrix (which has no variance here)
                        euclidean_dist_matrix = pairwise_distances(imputed_genotypes, metric='euclidean')
                        dist_df = pd.DataFrame(euclidean_dist_matrix, index=samples, columns=samples)
                        to_print.append("   Distance Matrix (all samples identical = distances near zero):")
                        #display(dist_df.round(2))
                    except Exception as dist_err:
                        to_print.append(f"   Could not calculate distances: {dist_err}")
                else:
                    to_print.append("   Cannot calculate distances (no variants processed).")

                # ===========================
                # === STEP 3: PCA (No Variance Case) ===
                # ===========================
                to_print.append("\n--- Principal Component Analysis (PCA) ---")
                to_print.append("   Skipping PCA calculation as no variance was detected.")
                # --- Placeholder PCA Plot ---
                to_print.append("\n--- PCA Plot (Placeholder - No Variance) ---")
                dummy_pca_data = {'SampleID': samples, 'PC1': np.zeros(n_samples), 'PC2': np.zeros(n_samples)}
                pca_df_dummy = pd.DataFrame(dummy_pca_data)
                to_print.append("   PCA Coordinates (N/A - No Variance):")
                #display(pca_df_dummy)
                try:
                    fig = px.scatter(
                        pca_df_dummy, x='PC1', y='PC2', hover_name='SampleID',
                        title=f"PCA Placeholder: No Variance Found (Whole genome)",
                        labels={'PC1': 'PC1 (N/A)', 'PC2': 'PC2 (N/A)'}, template='plotly_white'
                    )
                    fig.update_traces(marker=dict(size=8, opacity=0.8))
                    fig.update_layout(xaxis_range=[-1,1], yaxis_range=[-1,1], hovermode='closest')
                    fig.show(renderer="notebook") # Ensure correct renderer for your environment
                except Exception as plot_err:
                    to_print.append(f"   Could not generate placeholder PCA plot: {plot_err}")


                # ========================================================
                # === STEP 4: Phylogenetic Clustering (No Variance Case) ===
                # ========================================================
                # **Attempt clustering even with no variance, as distances are defined (zero)**
                to_print.append("\n--- Phylogenetic Clustering (Hierarchical Dendrogram) ---")
                to_print.append("   Attempting to generate dendrogram based on (near) zero distances...")
                to_print.append("   NOTE: Based on Euclidean distance computed from imputed genotypes (0,1,2).")

                # Check if enough samples and variants exist
                # Need at least 2 samples; variants don't need variance for distance=0
                if imputed_genotypes.shape[0] >= 2 and imputed_genotypes.shape[1] >= 1:
                    try:
                        # Attempt to create dendrogram using the zero-variance imputed data
                        fig_dendro = ff.create_dendrogram(
                            imputed_genotypes, # Use original imputed data (no variance)
                            orientation='left',
                            labels=samples,
                            linkagefun=lambda x: linkage(x, method='average', metric='euclidean') # Should result in zero-length branches
                        )
                        fig_dendro.update_layout(
                            title_text=f"Hierarchical Clustering Dendrogram<br>(UPGMA, Euclidean Distance - No Variance Detected)",
                            xaxis_title="Distance (Expected Near Zero)",
                            yaxis_title="Samples",
                            height=max(400, n_samples * 20),
                            margin=dict(l=max(100, max(len(s) for s in samples)*7)) # Adjust left margin for labels
                        )
                        fig_dendro.show(renderer="notebook") # Ensure correct renderer
                        to_print.append("   Dendrogram generated (should show all samples clustered with near-zero distance).")

                    except Exception as dendro_err:
                        to_print.append(f"   Warning: Could not generate dendrogram, possibly due to lack of variance or plotting library limitations: {dendro_err}")
                        # traceback.print_exc() # Uncomment for detailed traceback if needed
                else:
                    # This case (no variants) should have been caught earlier, but added for robustness
                    to_print.append("   Skipping dendrogram: Need at least 2 samples and 1 variant processed.")

            # --- Handle Case: SOME Variance Found ---
            else: # n_non_zero_variance > 0
                to_print.append("\n" + "="*50)
                to_print.append("RESULT: Variance detected. Proceeding with standard analysis.")
                to_print.append("="*50)

                if n_zero_variance > 0:
                    to_print.append(f"\nRemoving {n_zero_variance} zero-variance variants before proceeding.")
                    # Keep only columns (variants) with variance > 0
                    imputed_genotypes_filtered = imputed_genotypes[:, non_zero_var_indices]
                    # Update variant keys if needed for reporting (optional)
                    # final_variant_keys = [sorted_variant_keys[i] for i in non_zero_var_indices]
                else:
                    imputed_genotypes_filtered = imputed_genotypes # Use the original imputed matrix
                    # final_variant_keys = sorted_variant_keys # Use the keys after NaN filtering

                if imputed_genotypes_filtered.shape[1] == 0: # Final check
                     to_print.append(f"Error: No variants remaining after filtering zero-variance columns.")
                     return

                current_variants_used = imputed_genotypes_filtered.shape[1] # Number of variants used

                # ==============================================
                # === STEP 2: DISTANCE CALCULATION (Variance Case) ===
                # ==============================================
                to_print.append("\n--- Calculating Pairwise Distances ---")
                to_print.append(f"   (Calculating Euclidean distance on {current_variants_used} variants with non-zero variance)")
                try:
                    euclidean_dist_matrix = pairwise_distances(imputed_genotypes_filtered, metric='euclidean')
                    dist_df = pd.DataFrame(euclidean_dist_matrix, index=samples, columns=samples)
                    to_print.append("   Pairwise Euclidean Distance Matrix (rounded):")
                    #display(dist_df.round(2))
                except Exception as dist_err:
                    to_print.append(f"   Error calculating distances: {dist_err}")

                # --- PCA Data Preparation (Scaling) ---
                to_print.append("\nPreparing Data for PCA...")
                to_print.append("   Scaling data (centering and standardizing)...")
                scaler = StandardScaler()
                try:
                    # Ensure the filtered array is used here
                    scaled_genotypes = scaler.fit_transform(imputed_genotypes_filtered)
                    if np.isnan(scaled_genotypes).any():
                        to_print.append("Error: NaN values detected after scaling. Check imputation/data.")
                        # This might indicate a problem with the scaler or input data
                        return
                except Exception as scale_err:
                    to_print.append(f"   Error during scaling: {scale_err}")
                    traceback.print_exc()
                    return

                # ===========================
                # === STEP 3: PCA ===
                # ===========================
                to_print.append("\nPerforming Principal Component Analysis (PCA)...")
                n_samples_pca, n_variants_pca = scaled_genotypes.shape
                # Determine number of components (max 10, or limited by samples/variants)
                n_components = min(10, n_samples_pca, n_variants_pca)

                if n_variants_pca < 1: # Should have been caught, but double-check
                    to_print.append(f"Error: No variants available to perform PCA after all filtering.")
                    pca_success = False
                elif n_components < 1: # Handle cases like 1 sample or 1 variant
                    n_components = 1 # Calculate at least 1 PC if possible

                pca_success = False # Flag to check if PCA ran
                if n_variants_pca >= 1 and n_components >= 1:
                    to_print.append(f"   Running PCA with {n_components} components on {n_variants_pca} variants...")
                    try:
                        pca = PCA(n_components=n_components)
                        principal_components = pca.fit_transform(scaled_genotypes)
                        pca_success = True # Mark PCA as successful

                        # --- Prepare and Display PCA Results ---
                        to_print.append("\n--- PCA Results ---")
                        pc_cols = [f'PC{i+1}' for i in range(n_components)]
                        pca_df = pd.DataFrame(data=principal_components, columns=pc_cols)
                        pca_df['SampleID'] = samples
                        pca_df = pca_df[['SampleID'] + pc_cols]
                        explained_variance = pca.explained_variance_ratio_ * 100

                        to_print.append("   PCA Coordinates (rounded):")
                        #display(pca_df.round(4))
                        to_print.append("\n   Explained Variance (%):")
                        variance_df = pd.DataFrame({
                            'Principal Component': pc_cols,
                            'Explained Variance (%)': explained_variance.round(2)
                        })
                        #display(variance_df)

                        # --- Generate Interactive PCA Plot ---
                        to_print.append("\n--- Interactive PCA Plot ---")
                        if n_components >= 2:
                            fig_pca = px.scatter(
                                pca_df, x='PC1', y='PC2', hover_name='SampleID', hover_data=pc_cols,
                                title=f"PCA (Whole genome, {n_variants_pca} Variants used)",
                                labels={'PC1': f'PC1 ({explained_variance[0]:.2f}%)', 'PC2': f'PC2 ({explained_variance[1]:.2f}%)'},
                                template='plotly_white'
                            )
                            fig_pca.update_traces(marker=dict(size=8, opacity=0.8))
                            fig_pca.update_layout(hovermode='closest')
                            fig_pca.show(renderer="notebook") # Ensure correct renderer
                        elif n_components == 1:
                            to_print.append("   (Plotting PC1 only as only one component was calculated)")
                            pca_df['y_dummy'] = np.zeros(len(pca_df)) # Create dummy y-axis
                            fig_pca = px.scatter(
                                pca_df, x='PC1', y='y_dummy', hover_name='SampleID', hover_data=['PC1'],
                                title=f"PCA (Whole genome, {n_variants_pca} Variant) - PC1 Only",
                                labels={'PC1': f'PC1 ({explained_variance[0]:.2f}%)', 'y_dummy': ''},
                                template='plotly_white'
                            )
                            fig_pca.update_traces(marker=dict(size=8, opacity=0.8))
                            fig_pca.update_layout(hovermode='closest', yaxis_visible=False, yaxis_showticklabels=False)
                            fig_pca.show(renderer="notebook") # Ensure correct renderer

                    except Exception as pca_err:
                        to_print.append(f"   Error during PCA calculation or plotting: {pca_err}")
                        traceback.print_exc() # Uncomment for detailed traceback

                # ========================================================
                # === STEP 4: Phylogenetic Clustering (Variance Case) ===
                # ========================================================
                to_print.append("\n--- Phylogenetic Clustering (Hierarchical Dendrogram) ---")
                to_print.append("   NOTE: Using UPGMA ('average' linkage) based on Euclidean distance computed from imputed genotypes (0,1,2).")

                # Check if enough samples and variants exist for clustering
                if imputed_genotypes_filtered.shape[0] >= 2 and imputed_genotypes_filtered.shape[1] >= 1:
                    to_print.append(f"   Generating dendrogram for {n_samples} samples using {current_variants_used} variants...")
                    try:
                        # Use imputed data *before* scaling for dendrogram based on genotype similarity
                        fig_dendro = ff.create_dendrogram(
                            imputed_genotypes_filtered, # (samples x variants)
                            orientation='left',
                            labels=samples,
                            linkagefun=lambda x: linkage(x, method='average', metric='euclidean') # UPGMA
                        )
                        fig_dendro.update_layout(
                            title_text=f"Hierarchical Clustering Dendrogram<br>(UPGMA, Euclidean Distance, {current_variants_used} Variants)",
                            xaxis_title="Distance", yaxis_title="Samples",
                            height=max(400, n_samples * 20), # Adjust height based on samples
                            margin=dict(l=max(100, max(len(s) for s in samples)*7)) # Adjust left margin
                        )
                        fig_dendro.show(renderer="notebook") # Ensure correct renderer

                    except Exception as dendro_err:
                        to_print.append(f"   Error generating dendrogram: {dendro_err}")
                        # traceback.print_exc() # Uncomment for detailed traceback
                else:
                     to_print.append("   Skipping dendrogram: Need at least 2 samples and 1 variant with variance.")

            # --- End of the 'else' block for handling non-zero variance ---

            # Final completion message outside the if/else
            end_time = time.time()
            to_print.append("\n" + "="*50)
            to_print.append(f"Analysis run finished in {end_time - start_time:.2f} seconds.")
            to_print.append(f"Finished at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
            to_print.append("="*50)

        # --- General Error Handling ---
        except ImportError as ie:
            to_print.append(f"Import Error: {ie}. Please ensure required libraries are installed:")
            to_print.append("   pip install jupyter voila ipywidgets numpy pandas scikit-learn plotly PyVCF scipy ipython") # Added ipython for display
        except KeyError as ke:
             to_print.append(f"Key Error: {ke}. Problem accessing data, check dictionary structure or sample names.")
             traceback.print_exc()
        except ValueError as ve:
            to_print.append(f"Value Error during processing: {ve}")
            to_print.append("   Check VCF format, region, numeric conversion, or data consistency.")
            traceback.print_exc()
        except MemoryError as me:
            to_print.append(f"Memory Error: {me}. The dataset for the selected region might be too large.")
            to_print.append("   Try a smaller region or run on a machine with more RAM.")
        except Exception as e:
            to_print.append(f"An unexpected error occurred: {e}")
            to_print.append("\n--- Full Traceback ---")
            traceback.print_exc() # Print detailed traceback
        print("\n".join(to_print))

In [61]:
cluster_button = widgets.Button(
    description='Create clusters',
    disabled=False,
    button_style='success',
)

cluster_button.on_click(run_pca_analysis)

In [62]:
cluster_all_button = widgets.Button(
    description='Create clusters on whole genome',
    disabled=False,
    button_style='success',
)

cluster_all_button.on_click(run_pca_analysis_on_whole_genome)

In [63]:

file_inputs = widgets.VBox([
    widgets.Label("Input Files:"),
    reference_uploader,
    vcfs_uploader
], layout=widgets.Layout(margin='0 0 10px 0'))


region_selection = widgets.VBox([
    widgets.Label("Genomic Region:"),
    contig_dropdown,
    widgets.HBox([start_pos_widget, end_pos_widget])
], layout=widgets.Layout(margin='0 0 10px 0'))

action_buttons = widgets.HBox([
    calculate_button,
    cluster_button,
    cluster_all_button
], layout=widgets.Layout(margin='0 0 10px 0')) # Add some space below

ui = widgets.VBox([
    file_inputs,
    region_selection,
    action_buttons,
    widgets.HTML("<hr>"), # Add a visual separator
    output_area
])

display(ui)